# Metrics Analysis
This Kotlin notebook visualizes BLEU, ROUGE-L, METEOR, TokenF1, and chrF metrics from the CSV exported by the app.

Instructions:
1. Generate the CSV from the JSONL results: run the app with CSV mode. For example:
   - From IDE: Run Main with program arguments: `--csv src/javabest_no_select.jsonl metrics.csv`
   - Or from command line (if Gradle is set up): `./gradlew run --args="--csv src/javabest_no_select.jsonl metrics.csv"`
2. Place `metrics.csv` in the project root (or adjust the path below).


In [ ]:
// This cell loads the plotting library for Kotlin Notebook. If you run in Kotlin Jupyter, this enables Lets-Plot.
%use lets-plot


In [ ]:
import java.io.File

data class Row(val index:Int, val BLEU:Double, val ROUGE_L:Double, val METEOR:Double, val TokenF1:Double, val chrF:Double)

fun readCsv(path:String): List<Row> {
    val f = File(path)
    require(f.exists()) { "CSV not found at: $path. Please run the app in --csv mode to generate it." }
    return f.readLines().drop(1)
        .filter { it.isNotBlank() }
        .map { line ->
            val parts = line.split(',')
            Row(parts[0].toInt(), parts[1].toDouble(), parts[2].toDouble(), parts[3].toDouble(), parts[4].toDouble(), parts[5].toDouble())
        }
}

val csvPath = "metrics.csv" // change if needed, e.g., "src/metrics.csv"
val rows = readCsv(csvPath)
println("Loaded ${rows.size} rows from $csvPath")


In [ ]:
import org.jetbrains.letsPlot.*
import org.jetbrains.letsPlot.geom.*
import org.jetbrains.letsPlot.label.*
import org.jetbrains.letsPlot.themes.*

val df = mapOf(
    "BLEU" to rows.map { it.BLEU },
    "ROUGE_L" to rows.map { it.ROUGE_L },
    "METEOR" to rows.map { it.METEOR },
    "TokenF1" to rows.map { it.TokenF1 },
    "chrF" to rows.map { it.chrF }
)

fun avg(xs: List<Double>) = if (xs.isEmpty()) 0.0 else xs.sum() / xs.size
println("Averages:\nBLEU=${avg(df["BLEU"]!!)}\nROUGE_L=${avg(df["ROUGE_L"]!!)}\nMETEOR=${avg(df["METEOR"]!!)}\nTokenF1=${avg(df["TokenF1"]!!)}\nchrF=${avg(df["chrF"]!!)}")


## Histograms
Run each cell to see the distributions.


In [ ]:
val pBleu = ggplot(df) + geomHistogram { x = "BLEU" } + ggtitle("BLEU distribution")
pBleu


In [ ]:
val pRouge = ggplot(df) + geomHistogram { x = "ROUGE_L" } + ggtitle("ROUGE-L distribution")
pRouge


In [ ]:
val pMeteor = ggplot(df) + geomHistogram { x = "METEOR" } + ggtitle("METEOR distribution")
pMeteor


In [ ]:
val pF1 = ggplot(df) + geomHistogram { x = "TokenF1" } + ggtitle("TokenF1 distribution")
pF1


In [ ]:
val pChrF = ggplot(df) + geomHistogram { x = "chrF" } + ggtitle("chrF distribution")
pChrF


## Scatter plots
Relationships between metrics.


In [ ]:
val dfXY = mapOf(
    "BLEU" to df["BLEU"]!!,
    "METEOR" to df["METEOR"]!!,
    "ROUGE_L" to df["ROUGE_L"]!!,
    "TokenF1" to df["TokenF1"]!!
)
val pMeteorVsBleu = ggplot(dfXY) + geomPoint { x = "BLEU"; y = "METEOR" } + ggtitle("METEOR vs BLEU")
pMeteorVsBleu


In [ ]:
val pRougeVsBleu = ggplot(dfXY) + geomPoint { x = "BLEU"; y = "ROUGE_L" } + ggtitle("ROUGE-L vs BLEU")
pRougeVsBleu
